<a href="https://colab.research.google.com/github/drorb1987/wsc/blob/main/wsc_project_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
from torchvision.transforms import transforms
from google.colab import drive, files

ModuleNotFoundError: No module named 'google'

In [ ]:
files.upload()

In [4]:
!unzip wsc_dataset.zip

Archive:  wsc_dataset.zip
  inflating: readme.md               
   creating: test/
   creating: test/Angle/
  inflating: test/Angle/20181121-140024_200.jpg  
  inflating: test/Angle/20190310-140201_18_1080.jpg  
  inflating: test/Angle/20190310-140259_22_288.jpg  
  inflating: test/Angle/20190310-140416_26_384.jpg  
  inflating: test/Angle/20190310-140920_43_672.jpg  
  inflating: test/Angle/20190310-141318_57_516.jpg  
  inflating: test/Angle/20190310-142520_98_444.jpg  
  inflating: test/Angle/20190310-142759_107_348.jpg  
  inflating: test/Angle/20190310-142911_111_684.jpg  
  inflating: test/Angle/4BA51BAE-0B08-4A64-891F-137E1DB14EC2_frame_24.jpg  
   creating: test/Box/
  inflating: test/Box/16E5F68A-B304-4FA7-852F-3991E2B01589_frame_35.jpg  
  inflating: test/Box/1E09253D-4152-43BD-9F00-7737138B8034_frame_30.jpg  
  inflating: test/Box/30FFB41F-8B2B-4DDA-B194-D93519CAD127_frame_16.jpg  
  inflating: test/Box/3C144DBA-96C6-407F-867B-FACE0E4D8F8A_frame_27.jpg  
  inflating: test/Bo

In [2]:
from mobilenet_v2 import MobileNetV2

CLASSES = ['Angle', 'Box', 'Circle', 'Closeup', 'Crowd', 'Other']
model = MobileNetV2(num_classes=len(CLASSES))

In [ ]:
# Calculating mean and std on the dataset


# transform = transforms.Compose([transforms.Resize((720, 1280)), transforms.ToTensor()])
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
training_dataset = torchvision.datasets.ImageFolder(root='./train/', transform=transform)
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=len(training_dataset), num_workers=0)

dataiter = iter(training_loader)
images, _ = dataiter.next()
imgs = images.view(images.size(0), images.size(1), -1)
mean = imgs.mean(2).sum(0) / imgs.size(0)
std = imgs.std(2).sum(0) / imgs.size(0)
print(f"The mean is: {mean}")
print(f"The std is: {std}")

# mean = torch.tensor([0.3827, 0.4044, 0.3235])
# std = torch.tensor([0.1845, 0.1861, 0.1851])

In [7]:
# mean and std for (720, 1280) size
# mean = torch.tensor([0.3827, 0.4044, 0.3235])
# std = torch.tensor([0.1845, 0.1861, 0.1851])

# mean and std for (224, 224) size
mean = torch.tensor([0.3828, 0.4044, 0.3236])
std = torch.tensor([0.1719, 0.1736, 0.1733])

normalized_transform = transforms.Compose([
    # transforms.Resize([720, 1280]),
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    ])

batch_size = 6
normalized_training_dataset = torchvision.datasets.ImageFolder(root='./train/', transform=normalized_transform)
normalized_testing_dataset = torchvision.datasets.ImageFolder(root='./test/', transform=normalized_transform)
normalized_training_loader = torch.utils.data.DataLoader(normalized_training_dataset, batch_size=batch_size, shuffle=True)
normalized_testing_loader = torch.utils.data.DataLoader(normalized_testing_dataset, batch_size=batch_size, shuffle=False)

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [14]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(normalized_training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data
        
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        
        # Make predictions for this batch
        outputs = model(inputs)
        
        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()
        
        # Adjust learning weights
        optimizer.step()
        
        # Gather data and report
        running_loss += loss.item()
        if i % 5 == 4:
            last_loss = running_loss / 5 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(normalized_training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
        # inputs.detach()
        # labels.detach()
        # outputs.detach()
            
    return last_loss

In [10]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [11]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/soccer_trainer_{}'.format(timestamp))
epoch_number = 0

In [ ]:
EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    
    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    
    # We don't need gradients on to do reporting
    model.train(False)
    
    running_vloss = 0.0
    for i, vdata in enumerate(normalized_testing_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss
        # vinputs.detach()
        # vlabels.detach()
        
    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    
    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()
    
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
    
    epoch_number += 1

EPOCH 1:
  batch 5 loss: 1.8749531984329224
